In [ ]:
# pip in

In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [2]:
PINECONE_API_KEY = "5816834b-05e9-4a05-97c1-d4e1a29cab8c"
PINECONE_API_ENV = "gcp-starter"

In [3]:
# Extract data from pdf
def load_pdf(data):
    loader = DirectoryLoader(data, 
                   glob = "*.pdf",
                   loader_cls = PyPDFLoader)
    document = loader.load()
    
    return document

In [7]:
extracted_data = load_pdf("../data")

In [8]:
# Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print("length of my chnunk: ", len(text_chunks))

length of my chnunk:  7020


In [10]:
# download embedding model
def download_huggingface_embedding():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = download_huggingface_embedding()

In [ ]:
embeddings

In [ ]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

In [ ]:
# # Initializing the Pinecone
import pinecone
from langchain_pinecone import PineconeVectorStore

import os

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
pinecone.Pinecone(api_key = PINECONE_API_KEY,
             environment = PINECONE_API_ENV)

index_name = "indexkey"

docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name = "indexkey")

In [ ]:
docsearch = PineconeVectorStore.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs = docsearch.similarity_search(query, k = 3)

print("Result: ", docs)

In [13]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [14]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

In [15]:
llm = CTransformers(model = "model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                   model_type="llama",
                   config={'max_new_tokens':512,
                          'temperature': 0.8})

In [16]:
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type ="stuff",
    retriever = docsearch.as_retriever(search_kwargs = {'k':2}),
    return_source_documents = True,
    chain_type_kwargs = chain_type_kwargs
)

In [ ]:
while True:
    user_input = input(f"Input Prompt: ")
    result = qa({"query" : user_input})
    print("Response : ", result["result"] )

Input Prompt:  "what is acne"


C:\Users\prvnb\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response :  
Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
